In [1]:
import ipynbname
import os, sys

notebook_path = ipynbname.path()
notebook_dir = os.path.dirname(notebook_path)
ROOT_PATH = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.insert(0, os.path.join(ROOT_PATH))

In [2]:
from gcta import GCTA
gcta = GCTA()

GCTA Python v0.1.0 initialized


In [ ]:
gcta._load_plink_files(
    bfile=f"{ROOT_PATH}/test/bfile/test.majref",)



> Load PLINK files
|   Loaded 3900 samples and 985 SNPs


In [13]:
from gcta.core.data_io import load_bed, load_afreq
g = load_bed(
    f"{ROOT_PATH}/test/bfile/test.majref.bed", 
    n_samples=3900, 
    n_snps=985)

afreq = load_afreq(f"{ROOT_PATH}/test/bfile/test.majref.afreq")
g

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [14]:
g.shape

(3900, 985)

In [27]:
from gcta.analysis.grm import std_indicator
het_std, hom_std = std_indicator(g, afreq["ALT_FREQS"].to_numpy())

In [49]:
# beta
import numpy as np
betas = np.random.multivariate_normal(
    mean=[0, 0], 
    cov=[[0.1/985, 0.1/985], [0.1/985, 0.4/985]], 
    size=985
)


In [51]:
p = het_std @ betas[:, 0] + hom_std @ betas[:, 1] + np.random.normal(0, 0.5, size=3900)
p = (p - np.mean(p)) / np.std(p)

In [52]:
df_p = gcta.FAM.copy()
df_p["pheno"] = p

df_p[["FID", "IID", "pheno"]].to_csv(
    f"{ROOT_PATH}/test/test.pheno", 
    sep="\t", 
    header=False, 
    index=False
)